In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics.pairwise import cosine_similarity

import keras
from tensorflow import keras
from keras import Sequential
from keras import layers
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.utils.vis_utils import plot_model
from tensorflow.keras import Sequential, Input
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout,SeparableConv2D, Activation, BatchNormalization, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG

import librosa
import librosa.display
import IPython.display as ipd

import eli5
from eli5.sklearn import PermutationImportance
from joblib import dump, load

import os
import warnings
warnings.filterwarnings('ignore')

In [155]:
def get_features(y, sr):
    
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    harmony, perceptr = librosa.effects.hpss(y) 
    tempo = librosa.beat.tempo(y=y, sr=sr)
    
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    mfccs_means = np.mean(mfccs.T, axis=0),
    mfccs_var = np.var(mfccs.T, axis=0)
   
    res = [np.mean(chroma_stft), np.var(chroma_stft), np.mean(rmse), np.var(rmse), np.mean(spec_cent), np.var(spec_cent), np.mean(spec_bw), np.var(spec_bw), np.mean(rolloff), np.var(rolloff), np.mean(zcr), np.var(zcr), np.mean(harmony), np.var(harmony), np.mean(perceptr), np.var(perceptr), tempo[0]]
    
    for i in range(0, 20):
        res.append(mfccs_means[0][i])
        res.append(mfccs_var[i])
    
    return res

In [154]:
def predict(f, model):
    y, s = librosa.load(f)
    y, _ = librosa.effects.trim(y)
    f = get_features(y, s)

    f = np.array(f)
    f = f.reshape(1, -1)

    probs =  model.predict_proba(np.array(f))
    classes = model.classes_

    print(classes)
    print(probs[0])
    print(classes[np.argmax(probs[0])])
    print(np.max(probs[0]))

    return classes[np.argmax(probs[0])], np.max(probs[0])

In [160]:
labels = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

model = load('./xgb.joblib')

label, proc = predict('./data/genres_original/classical/classical.00007.wav', model)





[0 1 2 3 4 5 6 7 8 9]
[9.0405725e-02 1.0944908e-01 2.2606042e-04 2.5589743e-01 2.7365121e-01
 3.0226151e-03 1.4541785e-01 6.6801449e-03 2.6545586e-02 8.8704281e-02]
4
0.2736512


In [ ]:
filename = './data/genres_original/rock/rock.0000{number}.wav'.format(number="00000")

labels = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

model = load('./rforest.joblib')


for label in labels:
    for i in range(10):
        filename = './data/genres_original/{genre}/{genre}.0000{number}.wav'.format(genre=label, number=i)
        pred, prob = predict(filename, model)
        if pred != label:
            print('predicted: {pred} with probability: {prob} for {filename}'.format(pred=pred, prob=prob, filename=filename))



In [214]:
y, s = librosa.load(f'./data/genres_original/blues/blues.00000.wav')
y, _ = librosa.effects.trim(y)


_, data_p = librosa.effects.hpss(y)


print(np.mean(data_p))
print(np.var(data_p))






-1.0618483e-05
0.005704438
